<a href="https://colab.research.google.com/github/00sakshi00/Face-Mask-Detection/blob/main/FaceMask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
!pip install kaggle

In [55]:
# configuring the path of Kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Importing Face Mask Dataset

In [55]:
!kaggle datasets download -d omkargurav/face-mask-dataset

In [55]:
# extracting the compessed Dataset
from zipfile import ZipFile
dataset = '/content/face-mask-dataset.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

In [55]:
!ls

**Importing** **the** **Dependencies**

In [9]:
import os #access files
import numpy as np #convert imgaes to arrays
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
from google.colab.patches import cv2_imshow
from PIL import Image 
from sklearn.model_selection import train_test_split

In [55]:
with_mask_files = os.listdir('/content/data/with_mask')
print(with_mask_files[0:5])
print(with_mask_files[-5:])

In [55]:
without_mask_files = os.listdir('/content/data/without_mask')
print(without_mask_files[0:5])
print(without_mask_files[-5:])

In [55]:
print('Number of with mask images:', len(with_mask_files))
print('Number of without mask images:', len(without_mask_files))


**Creating Labels for the two class of Images**

with mask --> 1

without mask --> 0

In [55]:
# create the labels
with_mask_labels = [1]*3725
without_mask_labels = [0]*3828

In [55]:
print(with_mask_labels[0:5])
print(without_mask_labels[0:5])

print(len(with_mask_labels))
print(len(without_mask_labels))

In [55]:
labels = with_mask_labels + without_mask_labels

print(len(labels))
print(labels[0:5])
print(labels[-5:])

In [55]:
# displaying with mask image
img = mpimg.imread('/content/data/with_mask/with_mask_2590.jpg')
imgplot = plt.imshow(img)
plt.show()

In [55]:
# displaying without mask image
img = mpimg.imread('/content/data/without_mask/without_mask_2925.jpg')
imgplot = plt.imshow(img)
plt.show()

**Image Processing**



*   Resize the Images
*   Convert the Imgaes to numpy arrays

In [55]:
#convert images to numpy arrays

with_mask_path = '/content/data/with_mask/' # /added in path

data=[]

for img_file in with_mask_files:
  image = Image.open(with_mask_path + img_file)
  image= image.resize((128,128)) #make all images of same size
  image= image.convert('RGB') #some images may be gray scale
  image= np.array(image)
  data.append(image) 



without_mask_path = '/content/data/without_mask/' 

for img_file in without_mask_files:
  image = Image.open(without_mask_path + img_file)
  image= image.resize((128,128))
  image= image.convert('RGB')
  image= np.array(image)
  data.append(image) 

In [55]:
type(data)
len(data)

In [55]:
data[0]

In [55]:
type(data[0])   # ndarray => n dimensional array

In [55]:
data[0].shape

In [55]:
# converting image list and label list to numpy arrays

X = np.array(data)
Y = np.array(labels)

In [55]:
type(X)
type(Y)

In [55]:
print(X.shape)
print(Y.shape)

**Train Test Split**

In [55]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [55]:
print(X.shape, X_train.shape, X_test.shape)

In [55]:
# scaling the data

X_train_scaled = X_train/255
X_test_scaled = X_test/255

In [55]:
X_train[0]

In [55]:
X_train_scaled[0]

**Building Convolution Neural Network**

In [31]:
import tensorflow as tf         #build neural network developed by google
from tensorflow import keras    #developed by facebook

In [55]:
num_of_classes = 2

model = keras.Sequential()

model.add(keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(128,128,3)))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))

model.add(keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))

model.add(keras.layers.Flatten()) # 2D data to single dimensional

model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(0.5))  # avoid overfitting

model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.5))

model.add(keras.layers.Dense(num_of_classes, activation='sigmoid')) # sigmoid for binary classification

In [45]:
# compile the neural network
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])

In [55]:
# training the neural network
history = model.fit(X_train_scaled, Y_train, validation_split=0.1, epochs=5)

**Evalution**

In [55]:
loss, accuracy = model.evaluate(X_test_scaled, Y_test)
print('Test Accuracy =', accuracy)

In [55]:
h = history

# plot the loss value
plt.plot(h.history['loss'], label='train loss')
plt.plot(h.history['val_loss'], label='validation loss')
plt.legend()
plt.show()

# plot the accuracy value
plt.plot(h.history['acc'], label='train accuracy')
plt.plot(h.history['val_acc'], label='validation accuracy')
plt.legend()
plt.show()

**Predictive System**

In [55]:
input_image_path = input('Path of the image to be predicted: ')

input_image = cv2.imread(input_image_path)   # cv2=> to display using colab

cv2_imshow(input_image)

input_image_resized = cv2.resize(input_image, (128,128))

input_image_scaled = input_image_resized/255

input_image_reshaped = np.reshape(input_image_scaled, [1,128,128,3]) #for only one image

input_prediction = model.predict(input_image_reshaped)

print(input_prediction) # gives probability

input_pred_label = np.argmax(input_prediction)

print(input_pred_label)

if input_pred_label == 1:
  print('The person in the image is wearing a mask')
else:
  print('The person in the image is not wearing a mask')